In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# Upload the CSV file
import re
import os
import pandas as pd

def read_folder(folder_path, texts_list):
  all_files = os.listdir(folder_path)

  for file_name in all_files:
    file = open(folder_path+file_name, 'r+', encoding = "ISO-8859-1")
    text_content = file.read()
    texts_list.append((text_content, file_name))
    file.close()

   
folder_path = '/content/drive/My Drive/Graduação/TFG/TFG/Corpus/'
texts_list = []
read_folder(folder_path, texts_list)

df = pd.DataFrame(texts_list, columns = ['text', 'file_name']) 
print(df['file_name'])     

0                                         Ubirajara.txt
1                                              Diva.txt
2                                    Papeis Avulsos.txt
3                                 A Condessa Vesper.txt
4     Historias e Tradicoes da Provincia de Minas Ge...
                            ...                        
76                                Ao Correr da Pena.txt
77                                          Senhora.txt
78                              Girandola de amores.txt
79                                     O Moco Loiro.txt
80                                       Encarnacao.txt
Name: file_name, Length: 81, dtype: object


In [4]:
# download pretrained GloVe word embeddings
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-03-20 14:14:32--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-03-20 14:14:32--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-03-20 14:14:32--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [5]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [6]:
df.head()

,text,file_name
0,\n\nUbira...,Ubirajara.txt
1,\n\n\n DIVA\n\n\n\n A\n ...,Diva.txt
2,Papéis Avulsos\n\nADVERTÊNCIA\nEste título de ...,Papeis Avulsos.txt
3,A CONDESSA VÉSPER\n\nAS MEMÓRIAS DE UM CONDENA...,A Condessa Vesper.txt
4,Histórias e tradições da Provínci...,Historias e Tradicoes da Provincia de Minas Ge...


In [ ]:
for text_df, file_name in zip(df['text'], df['file_name']):
  
  # split the the text in the articles into sentences
  sentences = []
  
  for s in [text_df]:
    sentences.append(sent_tokenize(s))  

  # flatten the list
  sentences = [y for x in sentences for y in x]
  sentences = sentences[:3000]

  # remove punctuations, numbers and special characters
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

  # make alphabets lowercase
  clean_sentences = [s.lower() for s in clean_sentences]

  nltk.download('stopwords')# one time executio

  from nltk.corpus import stopwords
  stop_words = stopwords.words('portuguese')

  # function to remove stopwords
  def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new


  # remove stopwords from the sentences
  clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

  # Extract word vectors
  word_embeddings = {}
  f = open('glove.6B.100d.txt', encoding='utf-8')
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      word_embeddings[word] = coefs
  f.close()

  sentence_vectors = []
  for i in clean_sentences:
    if len(i) != 0:
      v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
      v = np.zeros((100,))
    sentence_vectors.append(v)

  len(sentence_vectors)

  # similarity matrix
  sim_mat = np.zeros([len(sentences), len(sentences)])

  from sklearn.metrics.pairwise import cosine_similarity

  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i != j:
        sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

  import networkx as nx

  nx_graph = nx.from_numpy_array(sim_mat)
  scores = nx.pagerank(nx_graph)

  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

  print(ranked_sentences)

  # Specify number of sentences to form the summary
  sn = 50
  folder_path = '/content/drive/My Drive/Graduação/TFG/textos_resumidos10/'
  file = open(folder_path+file_name, 'w+', encoding = "ISO-8859-1")
  # Generate summary
  for i in range(sn):
    if i < 40:
      print('')
    else:
      print(ranked_sentences[i][1])
      file.write(ranked_sentences[i][1])
  
  file.close()
  

Find the original article here https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/